## Install Libraries

In [1]:
!pip install torchsummary
!pip install torchgeometry
%pip install -U segmentation-models-pytorch

  Obtaining dependency information for torchsummary from https://files.pythonhosted.org/packages/7d/18/1474d06f721b86e6a9b9d7392ad68bed711a02f3b61ac43f13c719db50a6/torchsummary-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for torchgeometry from https://files.pythonhosted.org/packages/a6/d6/3f6820c0589bc3876080c59b58a3bad11af746a7b46f364b1cde7972bd72/torchgeometry-0.1.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.8 MB/s eta 0:00:00
  Obtaining dependency information for segmentation-models-pytorch from https://files.pythonhosted.org/packages/54/dd/0a3417eebc791d1f60ba9949a2c6fcb406ba671c67042179fb270409b17b/segmentation_models_pytorch-0.3.4-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for huggingface-hub>=0.24.6 from https://files.pythonhosted.org/packages/60/bf/cea0b9720c32fa01b0c4ec4b16b9f4ae34ca106b202ebbae9f03ab98cd8f/huggingface_hub-0.26.2-py3-none-any.whl

## Import Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.transforms import ToTensor
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
from torchinfo import summary
import timm
import segmentation_models_pytorch as smp
import wandb

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-122f3caa-91ba-1f39-6a1a-d2f7a7ebda4d)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Read data

In [5]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
image_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
len(image_path)

1000

In [6]:
# import os
# import cv2
# import numpy as np

# def calculate_mean_image_size(image_dir):
#     widths = []
#     heights = []

#     for filename in os.listdir(image_dir):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
#             img_path = os.path.join(image_dir, filename)
#             image = cv2.imread(img_path)
#             if image is not None:
#                 h, w = image.shape[:2]
#                 heights.append(h)
#                 widths.append(w)
#             else:
#                 print(f"Warning: Unable to read image {img_path}")

#     mean_width = int(np.mean(widths))
#     mean_height = int(np.mean(heights))
#     median_width = int(np.median(widths))
#     median_height = int(np.median(heights))

#     print(f"Number of images: {len(widths)}")
#     print(f"Mean Width: {mean_width}, Mean Height: {mean_height}")
#     print(f"Median Width: {median_width}, Median Height: {median_height}")

#     return mean_width, mean_height, widths, heights

# # Example usage:
# image_directory = '/kaggle/input/bkai-igh-neopolyp/test/test'
# mean_w, mean_h, widths, heights = calculate_mean_image_size(image_directory)

In [7]:
# import matplotlib.pyplot as plt

# def plot_dimension_distribution(widths, heights):
#     plt.figure(figsize=(12, 5))

#     plt.subplot(1, 2, 1)
#     plt.hist(widths, bins=20, color='skyblue')
#     plt.title('Width Distribution')
#     plt.xlabel('Width')
#     plt.ylabel('Frequency')

#     plt.subplot(1, 2, 2)
#     plt.hist(heights, bins=20, color='salmon')
#     plt.title('Height Distribution')
#     plt.xlabel('Height')
#     plt.ylabel('Frequency')

#     plt.tight_layout()
#     plt.show()

# # After collecting widths and heights in the function above:
# plot_dimension_distribution(widths, heights)

In [8]:
mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
len(mask_path)

1000

In [9]:
class DatasetCustom(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [10]:
img_resize = (256, 256)

In [11]:
dataset = DatasetCustom(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= img_resize,
                             transform = None)

## Augmentation

In [12]:
class AugmentDataset(Dataset):
    def __init__(self, dataset, transform=None, length_multiplier=1):
        self.dataset = dataset
        self.transform = transform
        self.length_multiplier = length_multiplier
        
    def __getitem__(self, idx):
        actual_idx = idx % len(self.dataset)
        image, label = self.dataset[actual_idx]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label

    def __len__(self):
        return len(self.dataset) * self.length_multiplier

In [13]:
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [14]:
# Define the split sizes
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Define the batch size
batch_size = 32
# Randomly split the dataset
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

train_dataset = AugmentDataset(train_dataset, transform=train_transformation, length_multiplier=2)
val_dataset = AugmentDataset(val_dataset, transform=val_transformation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [15]:
len(train_dataset)

1600

## Define Model

In [16]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 376MB/s]


In [17]:
# checkpoint = torch.load('/kaggle/input/kaggle-notebook/model.pth')
# model.load_state_dict(checkpoint['model'])
# model.to(device)

In [18]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)    

In [19]:
del dataset
del train_dataset
del val_dataset

## Train

Define Dice loss function

In [20]:
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, num_classes=3, epsilon=1e-6):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon

    def forward(self, outputs, targets):
        outputs = F.softmax(outputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=self.num_classes).permute(0, 3, 1, 2).float()

        # Flatten the tensors
        outputs_flat = outputs.contiguous().view(-1, self.num_classes)
        targets_flat = targets_one_hot.contiguous().view(-1, self.num_classes)

        intersection = torch.sum(outputs_flat * targets_flat, dim=0)
        union = torch.sum(outputs_flat + targets_flat, dim=0)

        dice_loss = 1 - (2 * intersection + self.epsilon) / (union + self.epsilon)
        mean_dice_loss = dice_loss.mean()

        return mean_dice_loss

In [21]:
from tqdm import tqdm

model.to(device)
criterion_ce = nn.CrossEntropyLoss()
criterion_dice = DiceLoss(num_classes=3)
alpha = 0.5  # Weight for balancing the two losses
best_val_loss = 999

Make use of learning rate scheduler

In [22]:
import torch.optim as optim

# Assuming you've already defined your optimizer
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Initialize the scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',                 # We want to minimize the validation loss
    factor=0.5,                 # Reduce LR by a factor of 0.5
    patience=1,                 # Number of epochs with no improvement after which LR will be reduced
    verbose=True,               # Print a message when LR is updated
    threshold=0.0001,           # Threshold for measuring the new optimum
    threshold_mode='rel',       # Mode for threshold
    cooldown=0,                 # Number of epochs to wait before resuming normal operation after LR has been reduced
    min_lr=1e-6                 # Lower bound on the learning rate
)

# scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)

In [23]:
import wandb

wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "e02f7703b40a2b3e0ab4801d4cb1d86b3b7327a6",
)

wandb.init(
    project="PolypSegment",
    config={
        "model" : "Unet",
        "init_learning_rate": learning_rate,
        "batch_size": batch_size,
        "optimizer": "Adam",
        "loss_functions": ["CrossEntropy", "Dice"],
        "imsize": img_resize
    }
)

# Add the model to WandB
wandb.watch(model, log="all")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tu7pham7 (hustcollab). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241122_075914-9p2rqccj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rich-dream-39
wandb: ⭐️ View project at https://wandb.ai/hustcollab/PolypSegment
wandb: 🚀 View run at https://wandb.ai/hustcollab/PolypSegment/runs/9p2rqccj


[]

In [24]:
num_epochs = 50
epoch_bar = tqdm(total=num_epochs, desc='Total Progress')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss_ce = criterion_ce(outputs, labels)
        loss_dice = criterion_dice(outputs, labels)
        loss = alpha * loss_ce + (1 - alpha) * loss_dice

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
#         del images, labels, outputs, loss
#         torch.cuda.empty_cache()
        
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)

            val_loss += criterion_dice(outputs.float(),labels.long()).item()
            
#             del images, labels, outputs
#             torch.cuda.empty_cache()
            
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)
        
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader), 'Learning_rate' : current_lr})
epoch_bar.close()

Total Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Loss: 0.0582111004


Total Progress:   2%|▏         | 1/50 [01:45<1:25:49, 105.10s/it]

Epoch [2/50], Loss: 0.0452912205


Total Progress:   4%|▍         | 2/50 [03:12<1:15:48, 94.77s/it] 

...

Epoch [48/50], Loss: 0.0164593038


Total Progress:  98%|█████████▊| 49/50 [1:11:28<01:26, 86.90s/it]

Epoch [49/50], Loss: 0.0167659952


Total Progress: 100%|██████████| 50/50 [1:12:54<00:00, 87.50s/it]

Epoch [50/50], Loss: 0.0166224075


In [25]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet34",        
#     encoder_weights="imagenet",     
#     in_channels=3,                  
#     classes=3     
# )
checkpoint = torch.load('/kaggle/working/model.pth')
model.load_state_dict(checkpoint['model'])
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [26]:
!mkdir prediction

## Get Testset Prediction

In [27]:
model.eval()
for i in os.listdir("/kaggle/input/bkai-igh-neopolyp/test/test"):
    img_path = os.path.join("/kaggle/input/bkai-igh-neopolyp/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_h = ori_img.shape[0]
    ori_w = ori_img.shape[1]
    img = cv2.resize(ori_img, img_resize, interpolation=cv2.INTER_AREA)
    transformed = val_transformation(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model(input_img).squeeze(0).cpu().numpy().transpose(1, 2, 0)
    mask = cv2.resize(output_mask, (ori_w, ori_h), interpolation=cv2.INTER_CUBIC)
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("prediction/{}".format(i), mask_rgb)
    # Clear variables to free memory
    del img, input_img, output_mask, mask, mask_rgb
    torch.cuda.empty_cache()

In [28]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        # print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

## Inference

In [29]:
# from torch.jit import load
# model = UNet()
# optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# checkpoint = torch.load(pretrained_path)

In [30]:
# optimizer.load_state_dict(checkpoint['optimizer'])

In [31]:
# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in checkpoint['model'].items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# # load params
# model.load_state_dict(new_state_dict)